# 炼丹杂谈

其实我也不知道到底分享这种东西到底能讲些啥…如果说做paper reading或是专题报告的分享，我大概还能说的具体点。这儿就当先开个坑吧。

除了一些结构上具有开创意义的创新，大家都是小修小补，想发文章就找个差点的baseline飚一飚，打比赛就找最好的部件拼一拼开始加trick开始大炼丹。其实都无外乎网络加深提feature再大batch到小batch交替摩擦。loss不降？查查数据，没问题加几层。上大batch加lr再小batch磨一磨，反正有BN这种东西以后大batch满天飞。所以说系统找个啥炼丹心得可说还真不容易……回头系统整理下再重新写。

不过这倒是可以先谈个问题：你train的真的准么……为啥我问这个问题？你可以自己手写几个层，跟pytorch和tf封装好的层测测输出误差……其实很好奇，为啥pytorch调的cudnn，误差居然比tf大。我有点没搞懂到底谁有问题了。最神奇的还是跟手写的卷积都不一样……如果测试没问题，那还是tensorflow的更准一点，不排除因为浮点精度问题出现了累积误差，但是这确实也挺难搞懂的，回头有时间仔细看看pytorch的卷积怎么写的。

## training技巧

其实就实验科学的结果来说，这东西真的是个玄学。
下面先讲讲我自己的实验习惯，解释的话可能过段时间再详细开一篇聊聊。
首先我们讲一讲选模型的原则。首先如果需要深层次的特征，抛开任务，提特征的时候有人都会偏爱直接选择resnet之类的比较深比较大的网络，但是很多时候在试验的情况下发现loss一直不降，震荡，不收敛，这时候……
先看看数据集或是喂的数据有没有问题才是第一要务。先检查自己prepare的数据是否没问题，比如检测自己的crop是不是有问题，或是在其他的一些数据处理函数上可视化view一下。如果本身喂进网络的数据没有问题，检查一下数据集。如果样本数量本来就不多，可以考虑先用类似的小一点模型，比如resneXt置换成GoogLenet，设置好超参数。